<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
splits

[<PrefetchDataset shapes: ((300, 300, 3), ()), types: (tf.uint8, tf.int64)>,
 <PrefetchDataset shapes: ((300, 300, 3), ()), types: (tf.uint8, tf.int64)>,
 <PrefetchDataset shapes: ((300, 300, 3), ()), types: (tf.uint8, tf.int64)>]

In [5]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [6]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [7]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [8]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [9]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [13]:
!rm -rf logs

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 6s 224ms/step - loss: 0.6680 - accuracy: 0.5791 - val_loss: 0.6244 - val_accuracy: 0.7610
Epoch 2/10
26/26 [==============================] - 6s 219ms/step - loss: 0.6126 - accuracy: 0.6873 - val_loss: 0.5811 - val_accuracy: 0.7707
Epoch 3/10
26/26 [==============================] - 5s 206ms/step - loss: 0.5597 - accuracy: 0.7421 - val_loss: 0.7313 - val_accuracy: 0.5220
Epoch 4/10
26/26 [==============================] - 5s 206ms/step - loss: 0.4974 - accuracy: 0.7701 - val_loss: 0.4493 - val_accuracy: 0.8195
Epoch 5/10
26/26 [==============================] - 5s 201ms/step - loss: 0.4183 - accuracy: 0.8297 - val_loss: 0.5347 - val_accuracy: 0.6927
Epoch 6/10
26/26 [==============================] - 5s 209ms/step - loss: 0.3757 - accuracy: 0.8577 - val_loss: 0.3149 - val_accuracy: 0.8732
Epoch 7/10
26/26 [==============================] - 6s 213ms/step - loss: 0.3086 - accuracy: 0.8856 - val_loss: 0.2334 - val_accuracy: 0.9366
Epoch 

In [16]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 17840), started 0:01:02 ago. (Use '!kill 17840' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 5s - loss: 0.6819 - accuracy: 0.5474 - val_loss: 0.6755 - val_accuracy: 0.7707
Epoch 2/5

Epoch 00002: saving model to weights.02-0.65.h5
26/26 - 5s - loss: 0.6617 - accuracy: 0.6144 - val_loss: 0.6543 - val_accuracy: 0.8195
Epoch 3/5

Epoch 00003: saving model to weights.03-0.63.h5
26/26 - 5s - loss: 0.6398 - accuracy: 0.6740 - val_loss: 0.6281 - val_accuracy: 0.7415
Epoch 4/5

Epoch 00004: saving model to weights.04-0.59.h5
26/26 - 5s - loss: 0.6106 - accuracy: 0.7105 - val_loss: 0.5928 - val_accuracy: 0.6195
Epoch 5/5

Epoch 00005: saving model to weights.05-0.59.h5
26/26 - 5s - loss: 0.5722 - accuracy: 0.7409 - val_loss: 0.5872 - val_accuracy: 0.6244


In [19]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


26/26 - 6s - loss: 0.6732 - accuracy: 0.5608 - val_loss: 0.6624 - val_accuracy: 0.6732


In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 6s - loss: 0.6784 - accuracy: 0.5547 - val_loss: 0.6662 - val_accuracy: 0.7317
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 5s - loss: 0.6449 - accuracy: 0.6642 - val_loss: 0.6818 - val_accuracy: 0.5024


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [21]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 5s - loss: 0.6633 - accuracy: 0.5998 - val_loss: 0.7599 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 5s - loss: 0.6221 - accuracy: 0.6764 - val_loss: 0.5842 - val_accuracy: 0.7122
Epoch 3/50
26/26 - 5s - loss: 0.5545 - accuracy: 0.7774 - val_loss: 0.5220 - val_accuracy: 0.7268
Epoch 4/50
26/26 - 5s - loss: 0.4944 - accuracy: 0.8029 - val_loss: 0.4607 - val_accuracy: 0.8390
Epoch 5/50
26/26 - 5s - loss: 0.4087 - accuracy: 0.8650 - val_loss: 0.3707 - val_accuracy: 0.8390
Epoch 6/50
26/26 - 5s - loss: 0.3533 - accuracy: 0.8674 - val_loss: 0.2886 - val_accuracy: 0.8976
Epoch 7/50
26/26 - 5s - loss: 0.3093 - accuracy: 0.8942 - val_loss: 0.2105 - val_accuracy: 0.9512
Epoch 8/50
26/26 - 5s - loss: 0.2144 - accuracy: 0.9465 - val_loss: 0.1672 - val_accuracy: 0.9610
Epoch 9/50
26/26 - 5s - loss: 0.1795 - accuracy: 0.9489 - val_loss: 0.1628 - val_accuracy: 0.9512
Epoch 10/50
26/26 - 5s - loss: 0.1455 - accuracy: 0.9586 - val_loss: 0.1233 - val_accuracy: 0.9610
Epoch 11/50
26/26 -

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [22]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 6s 214ms/step - loss: 0.6710 - accuracy: 0.6010 - val_loss: 0.6412 - val_accuracy: 0.5902
Epoch 2/5
26/26 [==============================] - 5s 206ms/step - loss: 0.6070 - accuracy: 0.6740 - val_loss: 0.5641 - val_accuracy: 0.6732
Epoch 3/5
26/26 [==============================] - 5s 198ms/step - loss: 0.5632 - accuracy: 0.6995 - val_loss: 0.4920 - val_accuracy: 0.9073
Epoch 4/5
26/26 [==============================] - 5s 200ms/step - loss: 0.4846 - accuracy: 0.7981 - val_loss: 0.4657 - val_accuracy: 0.7268
Epoch 5/5
26/26 [==============================] - 5s 204ms/step - loss: 0.4309 - accuracy: 0.8224 - val_loss: 0.4945 - val_accuracy: 0.7171


In [23]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.600973,0.671693,0.590244,0.641219
1,1,0.673966,0.607468,0.673171,0.564120
2,2,0.699513,0.563555,0.907317,0.491954
3,3,0.798053,0.485711,0.726829,0.465723
4,4,0.822384,0.432050,0.717073,0.494518


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 1
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir=os.path.join("log_dir"))])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 5s 204ms/step - loss: 0.6724 - accuracy: 0.5754 - val_loss: 0.6535 - val_accuracy: 0.7122

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 5s 194ms/step - loss: 0.6423 - accuracy: 0.7007 - val_loss: 0.6580 - val_accuracy: 0.5317

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 5s 203ms/step - loss: 0.6255 - accuracy: 0.6764 - val_loss: 0.6337 - val_accuracy: 0.7317

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 5s 197ms/step - loss: 0.6174 - accuracy: 0.7445 - val_loss: 0.6355 - val_accuracy: 0.6780

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 5s 198ms/step - loss: 0.6132 -

In [11]:
%tensorboard --logdir log_dir

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 19212.

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 5s 205ms/step - loss: 0.6690 - accuracy: 0.5779 - val_loss: 0.6241 - val_accuracy: 0.7073
Epoch 2/50
25/26 [===========================>..] - ETA: 0s - loss: 0.6138 - accuracy: 0.6875
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 5s 200ms/step - loss: 0.6087 - accuracy: 0.6946 - val_loss: 0.6582 - val_accuracy: 0.5122
Epoch 3/50
26/26 [==============================] - 5s 205ms/step - loss: 0.5491 - accuracy: 0.7774 - val_loss: 0.5509 - val_accuracy: 0.8341
Epoch 4/50
26/26 [==============================] - 5s 206ms/step - loss: 0.5263 - accuracy: 0.8102 - val_loss: 0.5390 - val_accuracy: 0.8098
Epoch 5/50
26/26 [==============================] - 5s 210ms/step - loss: 0.5073 - accuracy: 0.8090 - val_loss: 0.5023 - val_accuracy: 0.8000
Epoch 6/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4885 - accuracy: 0.8163
Epoch 00006: ReduceLROnPlateau re

In [ ]:
%tensorboard --logdir log_dir